In [ ]:
#docker ps : list the running containers
#docker ps -a, -f, -l :  -f= filter, -l=last, -a=all : list all the containers
#docker ps -a -q : list all the containers with only container id
#docker ps -a -q -f status=exited : list all the containers with status=exited with only container id
#docker ps -f status=exited : list all the containers with status=exited

#docker images : list all the images
#docker images -q : list all the images with only image id
#docker images -f dangling=true : list all the images with dangling=true
#docker images -f dangling=true -q : list all the images with dangling=true with only image id


#docker start <container_id> : start the container with container id
#docker stop <container_id> : stop the container with container id
#docker restart <container_id> : restart the container with container id
#docker pause <container_id> : pause the container with container id

#docker logs <container_id> : fetch the logs of the container with container id
#docker logs -f <container_id> : fetch and follow the logs of the container with container id
#docker logs -f <container_id> --tail 10 : fetch and follow the last 10 logs of the container with container id
#docker logs -f <container_id> --since 1m : fetch and follow the logs of the container with container id since 1 minute
#docker logs -f <container_id> --until 1m : fetch and follow the logs of the container with container id until 1 minute

#docker build . : build the image from the Dockerfile in the current directory
#docker build -t <image_name> . : build the image with the name <image_name> from the Dockerfile in the current directory
#docker build -t <image_name> . --no-cache : build the image with the name <image_name> from the Dockerfile in the current directory without cache
# -t : tag the image with the name <image_name>

#docker run <image_name> : run the container from the image with the name <image_name>
#docker run -d <image_name> : run the container from the image with the name <image_name> in the background
#docker run -d -p <host_port>:<container_port> <image_name> : run the container from the image with the name <image_name> in the background and publish the container port to the host port
#docker run --rm <image_name> : run the container from the image with the name <image_name> and remove the container after it exits

#docker run -it <image_name> : run the container from the image with the name <image_name> in the interactive mode
#docker run -it <image_name> sh : run the container from the image with the name <image_name> in the interactive mode with shell


#docker start -a -i <container_id> : start the container with the container id in the interactive mode and attach the terminal

#docker rm <container_name> <container_name> : remove the container with the container_name
#docker rm -f <container_id> : remove the container with the container id forcefully

#docker rmi <image_name> : remove the image with the image name
#docker rmi -f <image_id> : remove the image with the image id forcefully
#docker image prune : remove all the unused images

#remove all the containers
#docker rm -f $(docker ps -a -q) #working powershell

#remove all the images
#docker rmi -f $(docker images -q) #working powershell # -q flag to print only the image id


#docker cp dummy/. <container_id>:/test  - copy the files from the dummy directory to the container in the /test directory #local to container
#docker cp <container_id>:/test/. dummy - copy the files from the container in the /test directory to the dummy directory #container to local

#docker build -t <image_name>:<tag> . : build the image with the name <image_name> and tag <tag> from the Dockerfile in the current directory
#docker run <image_name>:<tag> : run the container from the image with the name <image_name> and tag <tag>

#example 
#docker build -t myimage:1.0 .
#docker run myimage:1.0



In [ ]:
# list out the terns used in building the dockerfile

# FROM : base image
# RUN : run the command
# COPY : copy the files from the host to the container
# ADD : copy the files from the host to the container
# WORKDIR : set the working directory
# EXPOSE : expose the port
# CMD : run the command
# ENTRYPOINT : run the command
# VOLUME : create the volume
# ENV : set the environment variable
# ARG : set the argument
# LABEL : set the label
# USER : set the user
# HEALTHCHECK : set the healthcheck
# SHELL : set the shell
# STOPSIGNAL : set the stop signal
# ONBUILD : set the on build
# COPY --from : copy the files from the other image

#example for python dockerfile

# FROM python:3.7 #base image
# WORKDIR /app #set the working directory
# COPY . /app #copy the files from the host to the container
# RUN pip install -r requirements.txt #run the command
# CMD ["python", "app.py"] #run the command



In [ ]:
# Use an official Selenium base image with Chrome
FROM selenium/standalone-chrome:latest

# Set the working directory to /app
WORKDIR /app

# Install Python and pip
USER root
RUN apt-get update && \
    apt-get install -y python3 python3-pip && \
    rm -rf /var/lib/apt/lists/*

# Copy only the requirements file first to leverage Docker cache
COPY requirements.txt .

# Install Python dependencies
RUN pip install --trusted-host pypi.python.org -r requirements.txt

# Copy the entire local directory contents into the container at /app
COPY . .

# Change the permissions of the /app directory to allow writing
RUN chmod -R a+w /app

# Switch back to the selenium user
USER seluser

# Run the tests and send email


CMD ["sh", "-c", "pytest test_cases_for_ui/*.py --json=report.json; python3 send_email.py"]

#CMD ["sh", "-c", "pytest test_cases_for_ui/test_c_c.py --json=report.json --html=report.html --self-contained-html -s -v"]


In [ ]:
"""
Why Use USER root for Installation
Permissions: Installing system packages typically requires root privileges.
Command Execution: Ensures the apt-get update and apt-get install commands have the necessary permissions to execute.
Why Copy requirements.txt First
Leverage Docker Cache:
Caching dependencies allows for faster builds when only the application code changes.
If requirements.txt hasn't changed, Docker will reuse the cached layer.
Copy Application Code
Post-Dependencies Installation:
Ensures that copying the application code and installing dependencies are separate steps.
If your code changes frequently but dependencies do not, this avoids reinstalling dependencies with each build.
Setting Permissions and Switching Back
Set Permissions:
Ensures the /app directory is writable.
Necessary for the non-root user to write to this directory.
Switch Back to Non-Root User:
Security Best Practice: Running processes as a non-root user reduces the risk of running processes with unnecessary root privileges.
Enhances container security by limiting permissions of running processes.
"""